In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train_csv = pd.read_csv('../input/indonesian-landmark/train.csv')
train_csv.head(10)

In [ ]:
len(train_csv)

In [ ]:
len(train_csv['landmark_id'].unique())

In [ ]:
import matplotlib.image as img
import matplotlib.pyplot as plt

def show_img(file_name):
    base_path = '../input/indonesian-landmark/Train/'
    for path in os.listdir(base_path):
        for file in os.listdir(base_path+path):
            if file == file_name:
                image = img.imread('../input/indonesian-landmark/Train/'+path+'/'+file_name)
    plt.imshow(image)
    plt.show()

In [ ]:
show_img('img10.jpg')

In [ ]:

train_csv = pd.read_csv('../input/indonesian-landmark/train.csv')
train_csv.head(10)

# put .jpg into the file name
def add_txt(fn):
    return fn+'.jpg'

train_csv['id'] = train_csv['id'].apply(add_txt)



# choose those labels with more than 70 images, and choose the first 70 images of each label
# move every training files to the same folder
%cd /kaggle/working
if not os.path.exists('training'):
    os.mkdir('training')
if not os.path.exists('validation'):
    os.mkdir('validation')
# if not os.path.exists('testing'):
#     os.mkdir('testing')    

import shutil
import random

label_list = train_csv['landmark_id'].unique()
cnt = 0
final_label_list = []
base_path = '../input/indonesian-landmark/Train/'

for label in list(label_list): # label order by random
    file_list = list(train_csv['id'][train_csv['landmark_id']==label])
    if len(file_list) >= 70:
        final_label_list.append(label)
        if not os.path.exists('/kaggle/working/training/'+str(label)):
            os.mkdir('/kaggle/working/training/'+str(label))
        if not os.path.exists('/kaggle/working/validation/'+str(label)):
            os.mkdir('/kaggle/working/validation/'+str(label))
#         if not os.path.exists('/kaggle/working/testing/'+str(label)):
#             os.mkdir('/kaggle/working/testing/'+str(label))
        for file in file_list[:50]:  # 50 files for training
            for path in os.listdir(base_path):
                for file_name in os.listdir(base_path+path):
                    if file == file_name:
                        src = '../input/indonesian-landmark/Train/'+path+'/'+file
            dst = '/kaggle/working/training/'+str(label)+'/'+file
            if not os.path.exists(dst):
                shutil.copyfile(src, dst)
        for file in file_list[50:70]: # 20 files for validation
            for path in os.listdir(base_path):
                for file_name in os.listdir(base_path+path):
                    if file == file_name:
                        src = '../input/indonesian-landmark/Train/'+path+'/'+file
            dst = '/kaggle/working/validation/'+str(label)+'/'+file
            if not os.path.exists(dst):
                shutil.copyfile(src, dst)
#         for file in file_list[160:200]: # 40 files for testing
#             src = '/kaggle/input/landmark-recognition-2020/train/'+file[0]+'/'+file[1]+'/'+file[2]+'/'+file
#             dst = '/kaggle/working/testing/'+str(label)+'/'+file
#             if not os.path.exists(dst):
#                 shutil.copyfile(src, dst)
        cnt += 1
    if cnt == 100: # only need 100 labels
        break
# 20,000 files in total

In [ ]:
len(final_label_list)

In [ ]:
print(len(os.listdir('./training')), len(os.listdir('./validation')))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '/kaggle/working/training'
validation_dir = '/kaggle/working/validation'
# test_dir = '/kaggle/working/testing'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size = 32,
    class_mode='categorical',
    seed=42)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    batch_size = 32,
    class_mode='categorical',
    seed=42)

# test_generator = test_datagen.flow_from_directory(
#     test_dir,
#     target_size=(256, 256),
#     batch_size = 1,
#     class_mode='categorical',
#     seed=42)

In [ ]:
# test_generator.class_indices.keys()
train_generator.class_indices.keys()

In [ ]:
from tensorflow.keras.applications import MobileNetV2

conv_base = MobileNetV2(include_top=False,
                        weights="imagenet",
                        input_shape=(300, 300, 3)
)

conv_base.trainable = True

conv_base.summary()

In [ ]:
from keras.layers import Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, Flatten, Conv2D, Input
from keras.models import Sequential
from keras import optimizers
import tensorflow as tf

model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(18, activation='softmax'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Training

In [ ]:
history = model.fit(
    train_generator,
    epochs=100, 
    validation_data=validation_generator
)


In [ ]:
# plot the training results
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, '#21466C', label='Training acc')
plt.plot(epochs, val_acc, '#ff0051', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, '#21466C', label='Training loss')
plt.plot(epochs, val_loss, '#ff0051', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
scores = model.evaluate(test_generator)
print('loss:', scores[0])
print('accuracy:', scores[1])

In [ ]:
label_list = list(test_generator.class_indices.keys())
label_list[:10]

In [ ]:
y_pred = model.predict(test_generator)

In [ ]:
y_pred.shape

In [ ]:
max(y_pred[0])

In [ ]:
y_pred_ls = y_pred.tolist()

In [ ]:
model.save('./indonesia_landmark_model_MobileNetV2_50epoch.h5')

In [ ]:
LANDMARK_MODEL= "./saved_50epoch"
tf.saved_model.save(model, LANDMARK_MODEL)

In [ ]:
#ubah ke zip
!zip -r ./saved_50epoch.zip ./saved_50epoch

In [ ]:
from IPython.display import FileLink

FileLink(r'./indonesia_landmark_model_MobileNetV2_50epoch.h5')

In [ ]:
!ls ./

In [ ]:
y = []

for i in range(len(y_pred_ls)):
    max_value = max(y_pred_ls[i])
    max_index = y_pred_ls[i].index(max_value)
    y.append(max_index)
    
y = np.array(y)

In [ ]:
scores = []
for i in range(len(y_pred_ls)):
    scores.append(max(y_pred[i]))
    
scores = np.array(scores)

In [ ]:
y

In [ ]:
scores

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y, scores, pos_label=0)

In [ ]:
y_0_1 = []
auc=[]
for j in y:
    if j != 10:
        y_0_1.append(0)
    else:
        y_0_1.append(1)
y_0_1 = np.array(y_0_1)
auc_of_the_label = roc_auc_score(y_0_1, scores)
auc.append(auc_of_the_label)
auc

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

auc=[]

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC')

for i in range(100):
    fpr, tpr, _ = roc_curve(y, scores, pos_label=i)
    plt.plot(fpr, tpr, label=i)
    
    y_0_1 = []
    for j in y:
        if j != i:
            y_0_1.append(0)
        else:
            y_0_1.append(1)
    y_0_1 = np.array(y_0_1)
    auc_of_the_label = roc_auc_score(y_0_1, scores)
    auc.append(auc_of_the_label)


plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC')

plt.show()



In [ ]:
scores

In [ ]:
max(auc)

In [ ]:
min(auc)

In [ ]:
label_list = list(test_generator.class_indices.keys())

In [ ]:
label_list[:10]

In [ ]:
def to_original_label(new_label): # 從新的label對應回原本資料的label
    return int(label_list[new_label])

In [ ]:
max_auc_label = to_original_label(auc.index(max(auc))) # label with the highest AUC
min_auc_label = to_original_label(auc.index(min(auc))) # label with the lowest AUC
print('max auc label', max_auc_label)
print('min auc label', min_auc_label)

### Test by eyes

First, images with the highest AUC score

In [ ]:
file_list = train_csv[train_csv['landmark_id']==max_auc_label]['id']
file_list

In [ ]:
for file in file_list[:20]:
    show_img(file)

Then, images with the lowest AUC score

In [ ]:
file_list = train_csv[train_csv['landmark_id']==min_auc_label]['id']
file_list

In [ ]:
for file in file_list[:20]:
    show_img(file)

Just view some beautiful images

In [ ]:
image_index = 0 # choose a image (0-3999)
image = test_generator[image_index][0] # [0][0]: the first 0 stands for the 0th image, the second 0 stands for the image array
image = image.reshape((256,256,3))
plt.imshow(image)
plt.show()
print(test_generator[0][1][0]) # array of the image's label

In [ ]:
def get_label(arr):
    for i in range(len(arr)):
        if arr[i] == max(arr):
            return label_list[i]

In [ ]:
# label from original data
get_label(test_generator[image_index][1][0])

In [ ]:
# label by prediction
get_label(model.predict(test_generator[image_index][0])[0])

In [ ]:
label = get_label(model.predict(test_generator[image_index][0])[0])
image_list = [] # image with the same label
for i in range(4000):
    if get_label(test_generator[i][1][0]) == label:
        image_list.append(i)


In [ ]:
if len(image_list) > 10:
    for i in range(10):
        if i != image_index:
            image = test_generator[image_list[i]][0]
            image = image.reshape((256,256,3))
            plt.imshow(image)
            plt.show()
        
else:
    for i in range(len(image_list)):
        if i != image_index:
            image = test_generator[image_list[i]][0]
            image = image.reshape((256,256,3))
            plt.imshow(image)
            plt.show()
